In [1]:
import cv2 as cv
import glob
import h5py
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import openslide
import os
import pandas as pd
import random
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
# Paths to raw TMA files.
PATH_TO_RAW_DATA = "/deep/group/aihc-bootcamp-fall2021/lymphoma/raw"
PATH_TO_IMAGES = os.path.join(PATH_TO_RAW_DATA, "svs")

# Path to processed data (extracted TMA patches)
PATH_TO_PROCESSED_DATA = "/deep/group/aihc-bootcamp-fall2021/lymphoma/processed"
PATH_TO_TMA_PATCHES = os.path.join(PATH_TO_PROCESSED_DATA, "tma_patches")

PATH_TO_TRAIN_DATA = os.path.join(PATH_TO_TMA_PATCHES, "train.hdf5")
PATH_TO_VAL_DATA = os.path.join(PATH_TO_TMA_PATCHES, "val.hdf5")
PATH_TO_TEST_DATA = os.path.join(PATH_TO_TMA_PATCHES, "test.hdf5")

# Path to annotations and labels (diagnoses)
PATH_TO_RAW_DATA = "/deep/group/aihc-bootcamp-fall2021/lymphoma/raw"
PATH_TO_ANNOTATIONS_CSV = os.path.join(PATH_TO_RAW_DATA, "cores")
PATH_TO_DIAGNOSES = os.path.join(PATH_TO_RAW_DATA, "core_labels.csv")

CASE = "CASE"
WHO_DIAGNOSIS = "2017 WHO DIAGNOSIS"
CLPA_DIAGNOSIS = "CLPA Diagnostic Bin"
LABEL = "label"
TMA_ID = "TMA ID"

In [3]:
# Get the list of paths to TMA svs files and TMA annotations csv files.
# We want the tma_slides_paths[i] to correspond to tma_annotations_paths[i] (hence, the calls to "sort").
tma_slides_paths = sorted(glob.glob(PATH_TO_IMAGES + "/*_TMA*.svs"), key= lambda s : s.split("_")[1])
tma_annotations_paths = sorted(glob.glob(PATH_TO_ANNOTATIONS_CSV + "/TMA*_annotations.csv"))

In [4]:
tma_names = ["TMA1", "TMA2", "TMA3", "TMA4", "TMA5", "TMA6A", "TMA6B", "TMA8"]
tma_ids = [1, 2, 3, 4, 5, 6, 6, 8]
tma_slides = [openslide.OpenSlide(tma_slide) for tma_slide in tma_slides_paths]
tma_annotations = [pd.read_csv(filename) for filename in tma_annotations_paths]

In [5]:
tma_case_to_diagnosis = pd.read_csv(PATH_TO_DIAGNOSES, delimiter=',')
tma_case_to_diagnosis[CLPA_DIAGNOSIS].value_counts()
tma_case_to_diagnosis.head()

,Unnamed: 0,TMA ID,CASE,2017 WHO DIAGNOSIS,CLPA Diagnostic Bin,label
0,0,1,E0001 B,NOT ON TMA,Excluded,-1
1,1,1,E0002 B,NON-DIAGNOSTIC,Excluded,-1
2,2,1,E0003 B,Classic Hodgkin Lymphoma,HL,1
3,3,1,E0004 B,"Follicular lymphoma, grade 1-2",FL,3
4,4,1,E0005 B,"Diffuse large B cell lymphoma, NOS",DLBCL,0


In [6]:
# Builds a dataframe that maps each patient ID in the annotated TMAs to the corresponding diagnosis and label.
def build_patient_to_label_df(tma_id, tma_annotations):
    print(tma_id)
    tma_patient_ids = sorted(list(set(tma_annotations["Name"])))
    tma_labels = pd.DataFrame()
    missing_patient_ids = set(["placenta", "tonsil"])
    clpa_diagnoses = []
    labels = []
    for patient_id in tma_patient_ids:
        if patient_id in missing_patient_ids: # Do not consider placenta/tonsisl.
            print(f"Could not find diagnosis for: {patient_id}")
            diagnosis = "Excluded"
            label = -1
        else:
            if not patient_id.rstrip()[-2].isspace() and patient_id.rstrip()[-1].isalpha():  # Add space between alphabet and number: "E0456B" -> "E0456 B"
                patient_id = patient_id[:-1] + " " + patient_id[-1]
            condition = (tma_case_to_diagnosis[CASE] == patient_id) & (tma_case_to_diagnosis[TMA_ID] == tma_id)
            tma_case_to_diagnosis_row = tma_case_to_diagnosis[condition]
            if len(tma_case_to_diagnosis_row[CLPA_DIAGNOSIS].values) == 0:
                print(f"Could not find diagnosis for: {patient_id}")
                diagnosis = "Excluded"
                label = -1
            else:
                diagnosis = tma_case_to_diagnosis_row[CLPA_DIAGNOSIS].values[0]
                label = tma_case_to_diagnosis_row[LABEL].values[0]
        clpa_diagnoses.append(diagnosis)
        labels.append(label)
    tma_labels["patient_id"] = tma_patient_ids
    tma_labels["clpa_diagnosis"] = clpa_diagnoses
    tma_labels["label"] = labels
    return tma_labels

tma_labels_df_list = [build_patient_to_label_df(tma_id, tma_annotation) for (tma_id, tma_annotation) 
                      in zip(tma_ids, tma_annotations)]

1
Could not find diagnosis for: placenta
Could not find diagnosis for: tonsil
2
Could not find diagnosis for: placenta
Could not find diagnosis for: tonsil
3
Could not find diagnosis for: placenta
Could not find diagnosis for: tonsil
4
Could not find diagnosis for: placenta
Could not find diagnosis for: tonsil
5
Could not find diagnosis for: placenta
Could not find diagnosis for: tonsil
6
Could not find diagnosis for: placenta
Could not find diagnosis for: tonsil
6
Could not find diagnosis for: E0137
Could not find diagnosis for: E0147
Could not find diagnosis for: E0184
Could not find diagnosis for: E0307
Could not find diagnosis for: placenta
Could not find diagnosis for: tonsil
8
Could not find diagnosis for: placenta
Could not find diagnosis for: tonsil


In [7]:
# Builds a dictionary from each label to the list of patient ids with that label
def build_label_to_patient_map(tma_labels_df):
    label_to_patients = dict()
    for label in range(0, 9):
        patient_ids_with_label = list(tma_labels_df[tma_labels_df[LABEL] == label]["patient_id"].values)
        label_to_patients[label] = patient_ids_with_label
    return label_to_patients

label_to_patients_list = [build_label_to_patient_map(tma_labels_df) for tma_labels_df in tma_labels_df_list]

In [8]:
# Splits the data into a train:val:test ratio of 0.8 : 0.1 : 0.2
# This method maintains this ratio for each diagnosis/label in the data. 
# For example, if we have 10 examples for label i: we randomly assign 8 to train, 1 to val, and 2 examples to test.
def get_train_test_splits(label_to_patients, train_fraction=0.8, val_fraction=0.1):
    train_patient_ids = []
    val_patient_ids = []
    test_patient_ids = []
    
    for label in label_to_patients:
        patient_ids = label_to_patients[label]
        if (len(patient_ids) <= 1): # Not enough patient IDs for this label to create a train/test split.
            train_patient_ids.extend(patient_ids)
        else:
            # We make two calls to train_test_split to split the patients into train : val : test splits.
            # 1) Call train_test_split to get train and test splits.
            train_ids, test_ids = train_test_split(patient_ids, train_size=train_fraction)
            if (len(train_ids) <= 1): # Not enough train patient IDs for this label to create a train/val split.
                train_patient_ids.extend(train_ids)
                test_patient_ids.extend(test_ids)
            else:
                # 2) Call train_test_split to split the train split into separate train and val splits.
                train_ids, val_ids = train_test_split(train_ids, train_size=(1 - (val_fraction / train_fraction)))
                train_patient_ids.extend(train_ids)
                val_patient_ids.extend(val_ids)
                test_patient_ids.extend(test_ids)
    return (train_patient_ids, test_patient_ids, val_patient_ids)

train_test_splits = [get_train_test_splits(label_to_patients) for label_to_patients in label_to_patients_list]

In [10]:
# Taken from DLBCL-Morph: extracting_roi_tma.ipynb.
# Takes in a patch (PIL image) and returns whether or not the patch is sufficiently non-white.
def is_patch_nonwhite_first(patch):
    PERCENT_WHITE_PIXELS_THRESHOLD = 0.95
    SAT_THRESHOLD = 0.05

    hsv_patch = matplotlib.colors.rgb_to_hsv(patch)
    saturation = hsv_patch[:,:,1]
    percent = np.mean(saturation < SAT_THRESHOLD)
    return percent <= PERCENT_WHITE_PIXELS_THRESHOLD

# Taken from DLBCL-Morph: extracting_roi_tma.ipynb.
# Takes in a patch (PIL image) and returns whether or not the patch is sufficiently non-white.
def is_patch_nonwhite_second(patch):
    GRAD_ZERO_THRESHOLD = 500
    gray = cv.cvtColor(np.array(patch), cv.COLOR_RGB2GRAY)
    sobelx = cv.Sobel(gray, cv.CV_64F, 1, 0, ksize=5)
    sobely = cv.Sobel(gray, cv.CV_64F, 0, 1, ksize=5)
    mag = np.abs(sobelx) + np.abs(sobely)
    return np.sum(mag == 0) <= GRAD_ZERO_THRESHOLD

# Extracts (default: 224 x 224) patches from a single core in the passed-in TMA.
# The tissue core is defined by: top-left: (xs, ys), bottom-right: (xe, ye).
# Only patches that are sufficiently non-white are returned.
# Returns an np.array of dimension: n x 224 x 224 x 3 (where n = # of returned patches)
def get_patches_from_core(tma, xs, ys, xe, ye, patch_size=224):
    patches = []
    for y in range(ys, ye, patch_size):
        for x in range(xs, xe, patch_size):
            patch = tma.read_region([x, y], 0, [patch_size, patch_size]).convert('RGB')
            if is_patch_nonwhite_first(patch) and is_patch_nonwhite_second(patch):
                patches.append(np.array(patch))
    
    if len(patches) == 0:
        return np.array([])
    return np.stack(patches)

def get_field_by_patient_id(tma_id, patient_id, field):
    missing_ids = set(["placenta", "tonsil"])
    
    if patient_id in missing_ids:
        print(f"Could not find {field} for: {patient_id}")
        return None
    
    elif not patient_id.rstrip()[-2].isspace() and patient_id.rstrip()[-1].isalpha():  # Add space between alphabet and number: "E0456B" -> "E0456 B"
        patient_id = patient_id[:-1] + " " + patient_id[-1]
    
    condition = (tma_case_to_diagnosis[CASE] == patient_id) & (tma_case_to_diagnosis["TMA ID"] == tma_id)
    
    if len(tma_case_to_diagnosis[condition][field].values) == 0:
        print(f"Could not find {field} for: {patient_id}")
        return None
    
    return tma_case_to_diagnosis[condition][field].values[0]

In [11]:
def tma_to_data_splits(tma_id, tma_annotations, tma_slide,
                       train_patient_ids, val_patient_ids, test_patient_ids,
                       train_f, val_f, test_f):
    patient_ids = set()
    patient_id_repeats = {}
    for index, row in tma_annotations.iterrows():
        patient_id = row["Name"]
        name = f"tma_{tma_id}_{patient_id}"
        
        who_diagnosis = get_field_by_patient_id(tma_id, patient_id, WHO_DIAGNOSIS)
        clpa_diagnosis = get_field_by_patient_id(tma_id, patient_id, CLPA_DIAGNOSIS)
        label = get_field_by_patient_id(tma_id, patient_id, LABEL)
        
        # Don't include patient IDs whose labels we either don't know (None)
        # or whose diagnosis is "excluded" (label = -1).
        if label == None or label < 0:
            continue
        
        if patient_id in train_patient_ids:
            f = train_f
        elif patient_id in val_patient_ids:
            f = val_f
        else:
            assert(patient_id in test_patient_ids)
            f = test_f
        
        # Deal with duplicate patients
        if (patient_id not in patient_ids):
            patient_id_repeats[patient_id] = 0
        patient_id_repeats[patient_id] += 1
        name += f"_v{patient_id_repeats[patient_id]}"
        
        xs, ys, width, height = int(row["X"]), int(row["Y"]), int(row["Width"]), int(row["Height"])
        xe, ye = xs + width, ys + height
        patches = get_patches_from_core(tma_slide, xs, ys, xe, ye)
        
        if patches.size == 0:
            print(f"No patches found for TMA: {tma_id}, Patient: {patient_id}")
            continue
        
        dset = f.create_dataset(name, data=patches, dtype='uint8')
        dset.attrs['tma_id'] = tma_id
        dset.attrs['patient_id'] = patient_id
        dset.attrs['who_diagnosis'] = who_diagnosis
        dset.attrs['clpa_diagnosis'] = clpa_diagnosis
        dset.attrs['label'] = label
        patient_ids.add(patient_id)
    
def build_train_test_splits(tma_ids, tma_slides, tma_annotations, train_test_splits):
    assert(len(tma_slides) == len(tma_annotations))
    train_f = h5py.File(PATH_TO_TRAIN_DATA, "w")
    val_f = h5py.File(PATH_TO_VAL_DATA, "w")
    test_f = h5py.File(PATH_TO_TEST_DATA, "w")
    for i in range(len(tma_slides)):
        tma_id = tma_ids[i]
        print(f"Building Train | Val | Test Splits for TMA: {tma_id}\n")
        (train_patient_ids, val_patient_ids, test_patient_ids) = train_test_splits[i]
        tma_to_data_splits(tma_id, tma_annotations[i], tma_slides[i],
                           train_patient_ids, val_patient_ids, test_patient_ids,
                           train_f, val_f, test_f)
    train_f.close()
    val_f.close()
    test_f.close()

build_train_test_splits(tma_ids, tma_slides, tma_annotations, train_test_splits)

Building Train | Val | Test Splits for TMA: 1

Could not find 2017 WHO DIAGNOSIS for: placenta
Could not find CLPA Diagnostic Bin for: placenta
Could not find label for: placenta
Could not find 2017 WHO DIAGNOSIS for: tonsil
Could not find CLPA Diagnostic Bin for: tonsil
Could not find label for: tonsil
Could not find 2017 WHO DIAGNOSIS for: tonsil
Could not find CLPA Diagnostic Bin for: tonsil
Could not find label for: tonsil
Could not find 2017 WHO DIAGNOSIS for: placenta
Could not find CLPA Diagnostic Bin for: placenta
Could not find label for: placenta
Building Train | Val | Test Splits for TMA: 2

Could not find 2017 WHO DIAGNOSIS for: tonsil
Could not find CLPA Diagnostic Bin for: tonsil
Could not find label for: tonsil
Could not find 2017 WHO DIAGNOSIS for: tonsil
Could not find CLPA Diagnostic Bin for: tonsil
Could not find label for: tonsil
Could not find 2017 WHO DIAGNOSIS for: placenta
Could not find CLPA Diagnostic Bin for: placenta
Could not find label for: placenta
Could 

## Testing

In [73]:
def get_num_examples_in_data_split(path_to_data_split):
    f = h5py.File(path_to_data_split, "r")
    patient_ids = sorted(set([key for key in f.keys() if key.endswith("_v1")]))
    f.close()
    return len(patient_ids)

def get_patient_ids_by_tma_and_data_split(tma_id, path_to_data_split):
    f = h5py.File(path_to_data_split, "r")
    patient_ids = set([key.split("_")[2] for key in f.keys() if key.startswith(f"tma_{tma_id}") and key.endswith("_v1")])
    f.close()
    return patient_ids

In [74]:
print("Number of training examples:", get_num_examples_in_data_split(PATH_TO_TRAIN_DATA))
print("Number of validation examples:", get_num_examples_in_data_split(PATH_TO_VAL_DATA))
print("Number of testing examples:", get_num_examples_in_data_split(PATH_TO_TEST_DATA))

Number of training examples: 386
Number of validation examples: 138
Number of testing examples: 78


In [78]:
tma_id = 1
tma_1_train_patient_ids = get_patient_ids_by_tma_and_data_split(tma_id, PATH_TO_TRAIN_DATA)
print(f"Number of training patient IDs from TMA {tma_id}:", len(tma_1_train_patient_ids))

expected_tma_1_train_patient_ids = train_test_splits[tma_id - 1][0]
print(f"Expected Number of training patient IDs from TMA {tma_id}:", len(expected_tma_1_train_patient_ids))

Number of training patient IDs from TMA 1: 43
Expected Number of training patient IDs from TMA 1: 43
